# Preprocessing Pipeline


The goal of the preprocessing pipeline here is to serve as a draft. Before we perform spike inference. This could be using oases or other more sophisticated method.


In [6]:
import numpy as np
import pandas as pd


from scipy.signal import savgol_filter, decimate
from scipy.ndimage import percentile_filter
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor


In [11]:
import utils as U
import importlib 
importlib.reload(U)

data  = U.load_data()

In [19]:
def isolate_sparse_epochs(data):
    """
    Return boolean mask over timepoints belonging to sparse-noise epochs.
    Works if stim_epoch_table is a (n_epochs,3) object array with one string column
    and two numeric columns (start_frame, end_frame).
    """
    t = data['t']
    mask = np.zeros_like(t, dtype=bool)
    epochs = data['stim_epoch_table']  # shape (n_epochs, 3)

    # detect which column is text
    first = epochs[0]
    name_col = next(i for i,x in enumerate(first) if isinstance(x, str))
    num_cols = [i for i,x in enumerate(first) if isinstance(x, numbers.Number)]
    if len(num_cols) != 2:
        raise ValueError(f"Expected 2 numeric cols, got {num_cols}")
    start_col, end_col = num_cols

    names  = epochs[:, name_col].astype(str)
    is_sp  = np.char.find(names, 'sparse') >= 0

    for row in epochs[is_sp]:
        start = int(row[start_col])
        end   = int(row[end_col])
        mask[start:end] = True

    return mask

def compute_qc_metrics(dff):
    """
    Compute per-cell ΔF/F variance and a simple SNR metric.
    Returns (variance array, snr array) of length n_cells.
    """
    var = np.var(dff, axis=1)
    p98 = np.percentile(dff, 98, axis=1)
    p02 = np.percentile(dff,  2, axis=1)
    noise_std = np.std(dff, axis=1)
    snr = (p98 - p02) / (noise_std + 1e-8)
    return var, snr
def estimate_neuropil_proxy(dff):
    """
    Simple proxy neuropil for each cell: mean of all other cells.
    Inputs:
      dff: (n_cells, n_time) ΔF/F traces
    Returns:
      neuropil: (n_cells, n_time)
    """
    total = np.sum(dff, axis=0, keepdims=True)
    neuropil = (total - dff) / (dff.shape[0] - 1)
    return neuropil

def regress_neuropil_robust(dff, neuropil, clamp=(0.5, 0.9)):
    """
    Perform per-cell robust regression of dff_i ~ ρ_i * neuropil_i + intercept,
    then subtract ρ_i * neuropil_i to obtain cleaned traces.

    Inputs:
      dff:      (n_cells, n_time) raw or sparse ΔF/F
      neuropil: (n_cells, n_time) neuropil proxy traces
      clamp:    (min, max) ρ value bounds

    Returns:
      cleaned:  (n_cells, n_time) neuropil-regressed traces
      coefs:    (n_cells,)    array of ρ_i values
    """

    n_cells = dff.shape[0]
    cleaned = np.zeros_like(dff)
    coefs   = np.zeros(n_cells)

    for i in range(n_cells):
        y = dff[i]
        X = neuropil[i][:, None]
        model = HuberRegressor().fit(X, y)
        rho = model.coef_[0]
        rho_clamped = np.clip(rho, clamp[0], clamp[1])
        cleaned[i] = y - rho_clamped * neuropil[i]
        coefs[i]   = rho_clamped

    return cleaned, coefs

def sliding_baseline(arr, t, window_sec=60, pct=10, mode="nearest"):
    """
    Compute and subtract a running-percentile baseline from each row of `arr`.

    Parameters
    ----------
    arr : array_like, shape (n_cells, n_time)
        The ΔF/F traces to baseline-correct.
    t : array_like, shape (n_time,)
        Time vector in seconds.
    window_sec : float
        Length of the sliding window in seconds.
    pct : float
        Percentile to use for baseline (e.g. 10).
    mode : str
        How to handle boundaries in scipy.ndimage.percentile_filter.

    Returns
    -------
    baseline : ndarray, shape (n_cells, n_time)
        The running baseline.
    corrected : ndarray, shape (n_cells, n_time)
        `arr - baseline`.
    win_frames : int
        Number of frames in the sliding window.
    """    

    dt = np.median(np.diff(t))
    win_frames = int(np.round(window_sec / dt))
    if win_frames % 2 == 0:
        win_frames += 1

    baseline = np.zeros_like(arr)
    for i in range(arr.shape[0]):
        baseline[i] = percentile_filter(arr[i], percentile=pct, size=win_frames, mode=mode)

    corrected = arr - baseline
    return baseline, corrected, win_frames

def smooth_dff_savgol(dff: np.ndarray, window: int, polyorder: int) -> np.ndarray:
    """
    Apply a Savitzky–Golay filter to each cell’s ΔF/F trace.

    Parameters
    ----------
    dff : array, shape (n_cells, n_time)
        The neuropil‐regressed, drift‐corrected ΔF/F traces.
    window : int
        Length of filter window (must be odd).
    polyorder : int
        Order of the polynomial fit (must be < window).

    Returns
    -------
    dff_smooth : array, shape (n_cells, n_time)
        The smoothed traces.
    """
    # enforce odd window
    if window % 2 == 0:
        raise ValueError("window must be odd")
    # apply filter along time axis=1
    return savgol_filter(dff, window_length=window, polyorder=polyorder, axis=1)

In [15]:
def isolate_sparse_epochs(data):
    """
    Return boolean mask over timepoints belonging to sparse-noise epochs.
    Works if stim_epoch_table is a (n_epochs,3) object array with one string column
    and two numeric columns (start_frame, end_frame).
    """
    t = data['t']
    mask = np.zeros_like(t, dtype=bool)
    epochs = data['stim_epoch_table']  # shape (n_epochs, 3)

    # detect which column is text
    first = epochs[0]
    name_col = next(i for i,x in enumerate(first) if isinstance(x, str))
    num_cols = [i for i,x in enumerate(first) if isinstance(x, numbers.Number)]
    if len(num_cols) != 2:
        raise ValueError(f"Expected 2 numeric cols, got {num_cols}")
    start_col, end_col = num_cols

    names  = epochs[:, name_col].astype(str)
    is_sp  = np.char.find(names, 'sparse') >= 0

    for row in epochs[is_sp]:
        start = int(row[start_col])
        end   = int(row[end_col])
        mask[start:end] = True

    return mask

def compute_qc_metrics(dff):
    """
    Compute per-cell ΔF/F variance and a simple SNR metric.
    Returns (variance array, snr array) of length n_cells.
    """
    var = np.var(dff, axis=1)
    p98 = np.percentile(dff, 98, axis=1)
    p02 = np.percentile(dff,  2, axis=1)
    noise_std = np.std(dff, axis=1)
    snr = (p98 - p02) / (noise_std + 1e-8)
    return var, snr

# 3.1 Drift removal

In [22]:
mask = isolate_sparse_epochs(data, target="locally_sparse_noise")
dff_sparse = data["dff"][:, mask]
t_sparse = data["t"][mask]
print(f"Sparse‑noise frames: {mask.sum()}")

var_raw, snr_raw = compute_qc_metrics(dff_sparse)

data.update(
    {
        "mask_sparse": mask,
        "dff_sparse": dff_sparse,
        "t_sparse": t_sparse,
        "var_raw": var_raw,
        "snr_raw": snr_raw,
    }
)

TypeError: isolate_sparse_epochs() got an unexpected keyword argument 'target'

In [18]:
# Step Neuropil Estimation & Robust Regression


# Grab all sparse‐noise ΔF/F traces
dff_sparse_all = data["dff_sparse"]  # shape: (n_cells, n_time)

# Estimate neuropil proxy (mean of other cells)
neuropil_proxy = estimate_neuropil_proxy(dff_sparse_all)

# Perform robust per‐cell regression & clamp ρ to [0.5, 0.9]
dff_regressed, rho_neuropil = regress_neuropil_robust(
    dff_sparse_all, neuropil_proxy, clamp=(0.0, 0.9)
)

# Store results for downstream steps
data.update(
    {
        "neuropil_proxy": neuropil_proxy,
        "dff_regressed": dff_regressed,
        "rho_neuropil": rho_neuropil,
    }
)

# 5) Quick sanity check
print(f"Neuropil regression done for {dff_regressed.shape[0]} cells.")
print("Example ρ values (first 5 cells):", rho_neuropil[:5])

KeyError: 'dff_sparse'

In [13]:
# Drift Removal via Sliding‐Window Percentile Baseline

# inputs: neuropil‐regressed ΔF/F and its timebase
dff_in = data["dff_regressed"]  # shape: (n_cells, n_time)
tvec = data["t_sparse"]  # shape: (n_time,)

# compute & subtract 10th‐percentile baseline over 60 s windows
baseline, dff_drift, win_frames = sliding_baseline(
    dff_in,
    tvec,
    window_sec=60,  # window length in seconds
    pct=10,  # percentile for baseline
    mode="nearest",  # boundary handling
)

# store for downstream steps
data.update(
    {
        "dff_baseline": baseline,
        "dff_drift": dff_drift,
        "drift_win": win_frames,
    }
)

# report
dt = np.median(np.diff(tvec))
print(f"Drift‐window: {win_frames} frames (~{win_frames*dt:.1f} s)")

KeyError: 'dff_regressed'

In [ ]:
# Savitzky–Golay Smoothing
from utils import smooth_dff_savgol

sg_window = 11  # frames
sg_poly = 2  # polynomial order

# pull the drift‑corrected traces
dff_dc = data["dff_drift"]  # shape (n_cells, n_time)

# smooth them
dff_smooth = smooth_dff_savgol(dff_dc, window=sg_window, polyorder=sg_poly)

# store back
data["dff_smooth"] = dff_smooth

print(f"Savitzky–Golay smoothing → window={sg_window}, polyorder={sg_poly}")
print("Smoothed shape:", dff_smooth.shape)